In [2]:
#!/usr/local/bin/python3
import rospy
from geometry_msgs.msg import PoseArray, Twist
from nav_msgs.msg import Odometry
import actionlib
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
import numpy as np
from scipy.spatial.transform import Rotation as R

In [3]:
class position():
    def __init__(self):
        self.x = 0
        self.y = 0
        self.z = 0
        self.ox = 0
        self.oy = 0
        self.oz = 0
        self.ow = 0


class husky_control:
    def __init__(self):
        self.ground_truth_sub = rospy.Subscriber('/ground_truth', Odometry, self.ground_truth_position_callback)
        self.vel_control_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)
        self.actionLib_server = actionlib.SimpleActionServer('husky_control', MoveBaseAction, self.actionLib_callback, False)
        self.position = position()

    def ground_truth_position_callback(self, data):
        self.position = data.pose.pose.position
        self.position.ox = data.pose.pose.orientation.x
        self.position.oy = data.pose.pose.orientation.y
        self.position.oz = data.pose.pose.orientation.z
        self.position.ow = data.pose.pose.orientation.w
        self.position.x = data.pose.pose.position.x
        self.position.y = data.pose.pose.position.y
        self.position.z = data.pose.pose.position.z

    def actionLib_callback(self, goal):
        vel_msg = Twist()
        while not self.arrived_at_goal(goal):
            vel_msg.linear.x = np.linalg.norm(np.array([self.position.x, self.position.y]) - np.array([goal.target_pose.pose.position.x, goal.target_pose.pose.position.y]))
            R = R.from_quat([self.position.ox, self.position.oy, self.position.oz, self.position.ow])
            yaw, _, _ = R.as_euler('zyx')
            vel_msg.angular.z = np.arctan2(goal.target_pose.pose.position.y - self.position.y, goal.target_pose.pose.position.x - self.position.x) - yaw
            self.vel_control_pub.publish(vel_msg)


    
    def arrived_at_goal(self, goal):
        if np.linalg.norm(np.array([self.position.x, self.position.y]) - np.array([goal.target_pose.pose.position.x, goal.target_pose.pose.position.y])) < 0.1:
            return True
        else:
            return False

        

In [ ]:

def main():
    rospy.init_node('husky_control', anonymous=True)
    husky = husky_control()
    rospy.spin()



if __name__ == '__main__':
    try:
        main()
    except rospy.ROSInterruptException:
        pass